In [0]:
import gzip
import shutil

def compress_file(file_path):
    compressed_file_path = f"{file_path}.gz"
    with open(file_path, 'rb') as f_in, gzip.open(compressed_file_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    print(f"File compressed to {compressed_file_path}")
    return compressed_file_path

# Example usage
compress_file("/Workspace/Users/lormana@campus.technion.ac.il/course_job_dict.json")


In [0]:
import os

def split_file(file_path, max_chunk_size):
    file_size = os.path.getsize(file_path)
    chunk_count = file_size // max_chunk_size + (1 if file_size % max_chunk_size > 0 else 0)
    
    with open(file_path, 'rb') as f:
        for i in range(chunk_count):
            chunk_data = f.read(max_chunk_size)
            with open(f"{file_path}_part_{i}", 'wb') as chunk_file:
                chunk_file.write(chunk_data)
    print(f"File split into {chunk_count} chunks.")

# Example usage
file_path = "/Workspace/Users/lormana@campus.technion.ac.il/course_job_dict.json"  # Replace with your file path
max_chunk_size = 10_485_760  # 10 MB in bytes

split_file(file_path, max_chunk_size)


## pip installs

In [0]:
%pip install translate

In [0]:
pip install google-cloud-translate

In [0]:
# Install the googletrans library
%pip install googletrans==4.0.0-rc1

In [0]:
%python
%pip install --upgrade googletrans==4.0.0-rc1

In [0]:
pip install spark-nlp


In [0]:
pip install transformers


In [0]:
!pip install sentence-transformers


In [0]:
pip install deep-translator

In [0]:
pip install wordcloud matplotlib

In [0]:
%restart_python or dbutils.library.restartPython()

### imports

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, udf

from googletrans import Translator
from deep_translator import GoogleTranslator

from sentence_transformers import SentenceTransformer

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from collections import Counter

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

import re
import nltk
import textwrap
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

## Translating the scraping

In [0]:
spark = SparkSession.builder.appName("Job Listings").getOrCreate()
file_path = "dbfs:/FileStore/tables/lormanat/job_listings_1.json"

schema = StructType([
    StructField("job_title", StringType(), True),
    StructField("job_description", StringType(), True),
    StructField("job_requirements", StringType(), True)
])

job_listings_df = spark.read.option("multiline", "true").schema(schema).json(file_path)

job_listings_df.printSchema()
display(job_listings_df)


In [0]:
filtered_df = job_listings_df.filter(
    col("job_title").rlike("(?i)data|דאטה|דאטא")
)

display(filtered_df)


In [0]:

%python
filtered_df.write.csv(
    "/dbfs/FileStore/tables/lormanat/data_jobs/",
    header=True,
    mode="overwrite"
)

In [0]:
print(os.listdir("/dbfs/FileStore/tables/lormanat/data_jobs/"))


In [0]:
from googletrans import Translator

def translate_udf(text):
    translator = Translator()
    return translator.translate(text, dest='en').text

translate = udf(translate_udf, StringType())

filtered_df = filtered_df.withColumn('job_title_translated', translate(filtered_df['job_title']))

display(filtered_df)

In [0]:
def translate_to_english(text):
    if text:
        try:
            return GoogleTranslator(source="auto", target="en").translate(text)
        except Exception as e:
            print(f"Error: {e}")
            return text
    return text

translate_udf = udf(translate_to_english, StringType())

translated_df = filtered_df.withColumn('job_title_translated', translate_udf(filtered_df['job_title'])) \
                           .withColumn('job_description_translated', translate_udf(filtered_df['job_description'])) \
                           .withColumn('job_requirements_translated', translate_udf(filtered_df['job_requirements']))

display(translated_df)


## The translated dataset

In [0]:
spark_df = spark.read.json("dbfs:/FileStore/tables/lormanat/translated_jobs.json")

display(spark_df)

In [0]:
job_descriptions = spark_df.select('job_description_translated').rdd.flatMap(lambda x: x).collect()
job_requirements = spark_df.select('job_requirements_translated').rdd.flatMap(lambda x: x).collect()
job_titles = spark_df.select('job_title_translated').rdd.flatMap(lambda x: x).collect()

## Translating syllabus

In [0]:
file_path = '/dbfs/FileStore/tables/lormanat/all_courses.csv'  # Replace with your file path

syllabus = pd.read_csv(file_path)

syllabus = syllabus.drop(columns=['Unnamed: 0','פקולטה'])

syllabus = syllabus.rename(columns={
    'מסגרת לימודים': 'study framework',
    'סילבוס': 'syllabus',
    'שם מקצוע':"subject's name"
})

display(syllabus)


In [0]:
columns_to_translate = [['study framework','syllabus',"subject's name"]]

translator = GoogleTranslator(source='hebrew', target='english')

for col in columns_to_translate:
    syllabus[col] = syllabus[col].apply(lambda x: translator.translate(x) if isinstance(x, str) else x)

output_file_path = 'translated_file.csv' 
syllabus.to_csv(output_file_path, index=False)

print("Translation complete. Translated file saved as:", output_file_path)


In [0]:
display(syllabus)

In [0]:
spark = SparkSession.builder.appName("PandasToSpark").getOrCreate()
syllabus_spark_df = spark.createDataFrame(syllabus)

syllabus_spark_df.show()

In [0]:
%python
from googletrans import Translator

def translate_udf(text):
    translator = Translator()
    return translator.translate(text, dest='en').text

translate = udf(translate_udf, StringType())

filtered_df_2 = (
    syllabus_spark_df
    .withColumn('study_framework_translated', translate(syllabus_spark_df['study framework']))
    .withColumn('syllabus_translated', translate(syllabus_spark_df['syllabus']))
    .withColumn('subject_name_translated', translate(syllabus_spark_df["subject's name"]))
)


In [0]:
syllabus_translated = filtered_df_2.select('study_framework_translated','syllabus_translated','subject_name_translated')
display(syllabus_translated)

### Translated syllabus

In [0]:
df_pandas = pd.read_csv('/dbfs/FileStore/tables/lormanat/syllabus_translated.csv')

translated_syllabus = spark.createDataFrame(df_pandas)

display(translated_syllabus)



In [0]:
courses = translated_syllabus.select('subject_name_translated').rdd.flatMap(lambda x: x).collect()
syllabuses = translated_syllabus.select('syllabus_translated').rdd.flatMap(lambda x: x).collect()

## BERT

In [0]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [0]:
example_syllabus = """Advanced methods for analyzing data and incorporating statistical tools and machine learning tools for data analysis, visually presenting them and building classification and prediction systems.Among the topics in the course will be taught: prediction and linear aggression, classification systems, Enihcam, Gnilpmaser, CAP, Rotcev Troppus, selection of models and vagulasis, decision trees and aggression, cluster analysis.Learning results: Theoretical understanding of the various methods and implementation of real data."""
example_syllabus_embeddings = sbert_model.encode([example_syllabus])

In [0]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [0]:
listing_embeddings = [sbert_model.encode(job_description) for job_description in job_requirements[:5]]
course_ranking = {}
for course, syllabus in zip(courses, syllabuses):
    course_ranking[course] = sum([cosine(listing_embedding, sbert_model.encode([syllabus])[0]) for listing_embedding in listing_embeddings])/5

sorted_courses = sorted(course_ranking, key=course_ranking.get, reverse=True)
print(sorted_courses[:10])
print(sorted_courses[-10:])
    

In [0]:
course_job_dict = {}
for course, syllabus in zip(courses, syllabuses):
    course_job_dict[course] = {}
    syllabus_embedding = sbert_model.encode([syllabus])[0]
    for job_title, job_requirement in zip(job_titles, job_requirements):
        if job_requirement is None:
            continue
        course_job_dict[course][job_title] = float(cosine(syllabus_embedding, sbert_model.encode([job_requirement])[0]))
#save dict to json
import json
with open('course_job_dict.json', 'w') as f:
    json.dump(course_job_dict, f)

In [0]:
selected_jobs = {}
for i, (job_description, job_title) in enumerate(zip(job_requirements, job_titles)):
    if job_description is None:
        continue
    if not i%100:
        print(i)
    sim = cosine(example_syllabus_embeddings, sbert_model.encode([job_description])[0])
    selected_jobs[(i, job_title)] = sim[0]
print({k: v for k, v in sorted(selected_jobs.items(), key=lambda item: item[1], reverse=True)[:10]})

### Courses from udemi and coursera

In [0]:
udemy_df = pd.read_csv('udemy.csv', encoding='ISO-8859-1')
coursera_df = pd.read_csv('coursera_courses.csv')

In [0]:
udemy_relevant_df = udemy_df[['name', 'description']]

In [0]:
udemy_relevant_df.columns = ['course', 'summary']
udemy_relevant_df['source'] = 'udemy'

In [0]:
coursera_relevant_df = coursera_df[['course_title', 'course_description']]

In [0]:
coursera_relevant_df.columns = ['course', 'summary']
coursera_relevant_df['source'] = 'coursera'

### Merged df

In [0]:
technion_relevant_df = translated_syllabus.toPandas()[['syllabus_translated', 'subject_name_translated']]
technion_relevant_df.columns = ['summary', 'course']
technion_relevant_df['source'] = 'technion'
technion_relevant_df.head()

In [0]:
merged_df = pd.concat([udemy_relevant_df, coursera_relevant_df, technion_relevant_df])
merged_df.head()
print(merged_df.size)

In [0]:
courses = list(merged_df['course'])
syllabuses = list(merged_df['summary'])
sources = list(merged_df['source'])

In [0]:
example_listing = """- Master's or PhD in Computer Science or related field
- Familiarity with Machine Learning algorithms
- Experience working as a Data Scientist for a product company
- Familiarity with Python/ Pandas
- Familiarity with Deep Learning algorithms on the following platforms: Tensorflow, Keras, Theano, CNTK, PyTorch"""
example_listing_embeddings = sbert_model.encode([example_listing])

In [0]:
%python
course_embeddings = [sbert_model.encode(str(summary)) for summary in syllabuses]

In [0]:
course_ranking = {}
for i, (course, embedding, source) in enumerate(zip(courses, course_embeddings, sources)):
    if i % 100 == 0:
        print(i)
    course_ranking[(course, source)] = np.dot(example_listing_embeddings, embedding)

sorted_courses = sorted(course_ranking, key=course_ranking.get, reverse=True)
print(sorted_courses[:10])
print(sorted_courses[-10:])

### NLP Analysis

In [0]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


In [0]:
def clean_text(text):
    text = text.lower() 
    text = re.sub(r'[^a-z\s]', '', text)
    return text

def tokenize(text):
    return word_tokenize(text)

def remove_stopwords(words):
    stop_words = set(stopwords.words("english"))
    return [word for word in words if word not in stop_words]

def lemmatize(words):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

#### Job listings' table

In [0]:
pandas_job_df = spark_df.toPandas()
pandas_job_df

In [0]:
pandas_job_df["cleaned_text"] = pandas_job_df["job_description_translated"].apply(clean_text)
pandas_job_df["tokens"] = pandas_job_df["cleaned_text"].apply(tokenize)
pandas_job_df["filtered_tokens"] = pandas_job_df["tokens"].apply(remove_stopwords)
pandas_job_df["lemmatized_tokens"] = pandas_job_df["filtered_tokens"].apply(lemmatize)
pandas_job_df["processed_text"] = pandas_job_df["lemmatized_tokens"].apply(lambda x: ' '.join(x))

pandas_job_df

#### Courses' table

In [0]:
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-z\s]', '', text)
    else:
        text = ''
    return text

merged_df["cleaned_text"] = merged_df['summary'].apply(clean_text)
merged_df["tokens"] = merged_df["cleaned_text"].apply(tokenize)
merged_df["filtered_tokens"] = merged_df["tokens"].apply(remove_stopwords)
merged_df["lemmatized_tokens"] = merged_df["filtered_tokens"].apply(lemmatize)
merged_df["processed_text"] = merged_df["lemmatized_tokens"].apply(lambda x: ' '.join(x))

#### Job titles we have

In [0]:
pandas_job_df

#### Clustering Job

In [0]:
syllabus_embeddings = sbert_model.encode([str(syllabus) for syllabus in syllabuses])

In [0]:
def generate_word_cloud(listings_input, title=None, removed_words=None, top_n=100):
    listing_embeddings = sbert_model.encode(listings_input)
    course_ranking = {}
    for course, syllabus_embedding in zip(courses, syllabus_embeddings):
        similarities = [cosine(listing_embedding, syllabus_embedding) for listing_embedding in listing_embeddings]
        course_ranking[course] = sum(similarities)

    course_ranking_list = course_ranking.tolist() if hasattr(course_ranking, 'tolist') else course_ranking
    sorted_courses_job = sorted(course_ranking_list, key=course_ranking.get, reverse=True)

    sorted_courses_job_str = ' '.join(sorted_courses_job[:top_n])

    if removed_words:
        sorted_courses_job_str = re.sub('('+r')|('.join(removed_words)+')', '', sorted_courses_job_str)

    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color='white',
        colormap='viridis'
    ).generate(sorted_courses_job_str)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  
    plt.title(f"{title if title else 'Word Cloud'}", fontsize=16)
    plt.show()

#### Clustering by Job Requirements

In [0]:
%python
job_requirements = pandas_job_df["job_requirements_translated"].fillna("") 

vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(job_requirements)  

range_n_clusters = range(2, 15)  
inertia_scores = []  
silhouette_scores = []  

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(X)
    
    inertia_scores.append(kmeans.inertia_)
    
    if n_clusters > 1:
        silhouette_avg = silhouette_score(X, clusters)
        silhouette_scores.append(silhouette_avg)

plt.figure(figsize=(8, 6))
plt.plot(range_n_clusters, inertia_scores, marker='o', label="Inertia (SSE)")
plt.title("Elbow Method: Optimal Number of Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia (SSE)")
plt.legend()
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(range(2, 15), silhouette_scores, marker='o', label="Silhouette Score", color="orange")
plt.title("Silhouette Scores: Optimal Number of Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Score")
plt.legend()
plt.show()

In [0]:
%python
pandas_job_df["job_requirements_translated"] = pandas_job_df["job_requirements_translated"].fillna("")

vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(pandas_job_df["job_requirements_translated"])

n_clusters = 6  
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
pandas_job_df["cluster"] = kmeans.fit_predict(X)

pca = PCA(n_components=2)
reduced_data = pca.fit_transform(X.toarray())

plt.figure(figsize=(8, 6))
for i in range(n_clusters):
    points = reduced_data[pandas_job_df["cluster"] == i]
    plt.scatter(points[:, 0], points[:, 1], label=f"Cluster {i}")

plt.title("Job requirements Clusters")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.legend()
plt.show()

In [0]:
cluster_counts = pandas_job_df['cluster'].value_counts()

print(cluster_counts)

In [0]:
grouped_clusters = pandas_job_df.groupby('cluster')

for cluster, group in grouped_clusters:
    print(f"Cluster {cluster}:")
    print(group['job_title_translated'].sample(n=5, random_state=42, replace=True)) 
    print("-" * 50)

In [0]:
random_job_requirements_2 = [list(pandas_job_df[pandas_job_df['cluster'] == i]['job_requirements_translated'].loc[pandas_job_df[pandas_job_df['cluster'] == i]['job_title_translated'].sample(n=10, random_state=42, replace=True).index]) for i in range(6)]

In [0]:
for i in range(6):
    if i not in [0, 4]:
        continue
    generate_word_cloud(
        random_job_requirements_2[i], 
        title="Cloud Engineering Cluster's Courses" if i == 0 else ("SQL Bootcamp Cluster's Courses" if i == 4 else f"Cluster {i}'s Courses"), 
        removed_words=['IBM', 'Python', 'Data Science', 'Data', 'Machine Learning', 'Fundamentals', 'Google Cloud'],
        top_n=50
        )


### Proof of Concept - feature demo

In [0]:
listings = [ ('Data Scientist for a start up in the cyber world ', """- Master's or PhD in Computer Science or related field
- Familiarity with Machine Learning algorithms
- Experience working as a Data Scientist for a product company
- Familiarity with Python/ Pandas
- Familiarity with Deep Learning algorithms on the following platforms: Tensorflow, Keras, Theano, CNTK, PyTorch"""),
('Junior data engineer at a tech company', """-3 years of experience in Java/Scala development
-Experience in Big Data technologies
-Spark
-Mastery of big data - Kinesis / Kafka / Cassandra / Solr
-Good at data engineering
-Experience with data visualization tools
-Experience with sklearn, numpy, pandas
-Experience with machine learning
-Experience with deep learning
-Experience with big data
-Experience with reinforcement learning
-Experience with pytorch pyspark nlp
""")]

In [0]:
display(spark_df)

In [0]:
listings = [('Java big data',"""At least 4 years of experience in Java development.
Experience in team management.
Experience in architecture and development from Scratch.
Experience working with NOSQL databases.
Experience in Big Data technologies.
B.sc/ BA in Computer Science/ Software Engineering."""),('Full Stack Software Engineer for a company dealing with the world of data in Herzliya',"""- 5 years of experience with NodeJS
- 4 years of experience with React or Vue
- Familiarity with Docker
- Familiarity with one of the following technologies: MongoDB/Elasticsearch/RabbitMQ/kafka""")]

In [0]:
max_chars_per_line = 35
def show_relevant_courses(listings):
    job_titles = [listing[0] for listing in listings]
    requirements = [listing[1] for listing in listings]
    requirements_embeddings = sbert_model.encode(requirements)
    similarities = {}
    for course, course_embedding, source in zip(courses, course_embeddings, sources):
        similarities[(course, source)] = (sum([cosine(course_embedding, job_embedding) for job_embedding in requirements_embeddings])/len(requirements), course_embedding)

    relevant_courses = sorted(similarities, key=lambda x: similarities[x][0], reverse=True)[:5]
    edges = []
    for i, (job, job_embedding) in enumerate(zip(job_titles, requirements_embeddings)):
        for j, (course, source) in enumerate(relevant_courses):
            edges.append((job, course, source, cosine(similarities[(course, source)][1], job_embedding)))

    fig = go.Figure()
    deafult_width = 1.2
    def add_rectangle(fig, x, y, text, width=deafult_width, height=0.4, color="blue", hovertext=None):
        text = "<br>".join(textwrap.wrap(text, width=max_chars_per_line, break_long_words=False))
        rect_x = [x - width/2, x + width/2, x + width/2, x - width/2, x - width/2]
        rect_y = [y - height/2, y - height/2, y + height/2, y + height/2, y - height/2]
        fig.add_trace(go.Scatter(
            x=rect_x, y=rect_y,
            fill="toself",
            mode="lines",
            line=dict(color=color),
            fillcolor=color,
            hoverinfo="none"
        ))
        fig.add_trace(go.Scatter(
            x=[x], y=[y],
            mode="text+markers",
            text=[text],
            textfont=dict(size=12, color="white"),
            hoverinfo="text" if hovertext else "none",
            hovertext=hovertext,
            marker=dict(opacity=0)
        ))

    # Add job nodes (left side)
    for i, (job, requirements) in enumerate(listings):
        add_rectangle(
            fig, x=-1, y=2*i + 1, text=job, color="rgb(58, 232, 161)",
            hovertext="<br>".join(requirements.split('-'))
        )

    source_to_color = {'technion': 'green', 'udemy': "rgb(164, 53, 240)", 'coursera': "rgb(0, 86, 210)"}
    # Add course nodes (right side)
    for i, (course, source) in enumerate(relevant_courses):
        add_rectangle(
            fig, x=1, y=4-i, text=course, color=source_to_color[source],
                            hovertext=f"Source: {source}"
                            )

    max_sim = max(sim for _, _, _, sim in edges)
    min_sim = min(sim for _, _, _, sim in edges)
    edges = [(job, course, source, 1/2+(sim - min_sim)/(max_sim-min_sim)/2) for job, course, source, sim in edges if sim > 0.5]

    # Add edges
    for job, course, source, sim in edges:
        dissim = 1 - sim
        color = f"rgb({dissim*255}, {dissim*255}, {dissim*255})"
        fig.add_trace(go.Scatter(
            x=[-1+deafult_width/2, 1-deafult_width/2],
            y=[job_titles.index(job)*2 + 1, 4-[name for name, sour in relevant_courses].index(course)],
            mode="lines",
            line=dict(width=5, color=color)
        ))

    # Update layout
    fig.update_layout(
        showlegend=False,
        xaxis=dict(showgrid=False, zeroline=False, visible=False),
        yaxis=dict(showgrid=False, zeroline=False, visible=False),
        margin=dict(l=100, r=100, t=50, b=50),
        plot_bgcolor="white",
        height=600,
        hoverlabel=dict(
            bgcolor="white",  # Background color
            font_size=12,     # Font size
            font_family="Arial"  # Font family
        )
    )

    # Display the figure
    fig.show()


In [0]:
show_relevant_courses(listings)